# EmoSonics: Emotion to Sound Experiment 

## Synthesis engines: SC3, MAX

In [1]:
%pylab
import ipywidgets as widgets
from functools import partial
from IPython.display import display
import pandas
import pickle, time

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [8]:
# first: select sound model 
#soundmodel = 'abstract'
soundmodel = 'vocal'
if (soundmodel == 'abstract'):
    portMax = 9000
else: portMax = 9001
# Here the abstract and physilogical sounds used two different address. This is to prevent cross
# triggering if two models are opened at the same time but only one is intended to be used. 
# vocal    (by TH) uses SC synth / server on port 57110
# abstract (by JJ) uses MAX/Msp on port 9000
# physiological (by JJ) uses MAX/Msp on port 9001

In [ ]:
#import imp
#pyson = imp.load_source('pyson', '/Users/thermann/Desktop/Inbox/pyson/pyson.py')

In [9]:
import imp
sonmod = imp.load_source('sonmod', 'EmoSonics-soundmodels.py')
# I moved it from .py to here to accomodate the non notebook version of rating
print "wait for SC to start..."
time.sleep(5); # wait until SC is started...
print "resume: establish OSC interfaces..."

OSError: [Errno 2] No such file or directory

In [5]:
import OSC, time, threading

clientSC  = OSC.OSCClient(); clientSC.connect(("127.0.0.1", 57110))

def sc_msg(onset, msgAdr="/s_new", msgargs=["s1", 2000, 1, 0, "freq", 300, "amp", 0.5]):
    global clientSC
    bundle = OSC.OSCBundle()
    msg = OSC.OSCMessage()
    msg.setAddress(msgAdr)
    msg.extend(msgargs)
    bundle.append(msg)
    bundle.setTimeTag(onset)
    clientSC.send(bundle)


clientMAX = OSC.OSCClient(); clientMAX.connect(("127.0.0.1", portMax)) 
def max_msg(onset, msgAdr="/s_new", msgargs=["freq", 300, "amp", 0.5]):
    global clientMAX
    bundle = OSC.OSCBundle()
    msg = OSC.OSCMessage()
    msg.setAddress(msgAdr)
    msg.extend(msgargs)
    bundle.append(msg)
    bundle.setTimeTag(onset)
    clientMAX.send(bundle)

# 2. Initialize Synthesizer
## For SC3
* make sure that SuperCollider is started
* the server is booted
* the synthdef is known: (here the synths for the vocalsyn)

(
SynthDef(\jj1, {|out=0, amp=1, pitch=50, chirp=0, dur=0.5, att=0.0, decslope=(-12),
	amint=0, amfreq=0, lfnfrq=0, lfnint=0, vowel=2, voweldiff=0, bright=1, pan=0 |

	var sig, sum, aenv, fenv, amsig, va, ve, vi, vo, vu, blend;

	amsig = SinOsc.kr(amfreq, mul: 0.5*amint, add: 0.5);
	aenv = Line.ar(0, decslope,dur, doneAction: 2).dbamp * EnvGen.kr(Env.new([0, 1, 1, 0], [att*dur, dur-(dur*att)-0.01, 0.01]));
	fenv = Line.kr(pitch, pitch+chirp, dur).midicps + LFNoise1.kr(lfnfrq, lfnint*pitch.midicps);
	vu = Vowel(\u, \tenor);
	vo = Vowel(\o, \tenor);
	va = Vowel(\a, \tenor);
    ve = Vowel(\e, \tenor);
	vi = Vowel(\i, \tenor);
	blend = Line.kr(vowel, vowel+voweldiff, dur);
	sig =  Formants.ar(fenv, vu
		.blend(vo, blend.linlin(0,1,0,1,\minmax))
		.blend(va, blend.linlin(1,2,0,1,\minmax))
		.blend(ve, blend.linlin(2,3,0,1,\minmax))
		.blend(vi, blend.linlin(3,4,0,1,\minmax))
		.brightenExp(bright.reciprocal, 1));
	sum = sig*amsig*aenv;
	Out.ar(0, Pan2.ar(sum, pan, amp));
}).add();

SynthDef(\reverb, {|outbus, mix=0.25, room=0.15, damp=0.5, amp=1.0|
	var sig;
	sig = In.ar(outbus, 2);
	ReplaceOut.ar(outbus,
		FreeVerb2.ar( sig[0], sig[1], mix, room, damp, amp));
}).add();
)


## For MAX/Msp: 
* start the AbstractModel.app, 
* so that MAX/Msp awaits OSC messages on port 9000

## 3. Synths initialization
* for the current synthesis this is merely starting the reverb, since the vocal synthesizer is spawned
* for a continuous playing synths, whose parameters are merely set/updated, you'd send the OSC message here to start playback / create the instance

In [6]:
if (soundmodel == "vocal") :    
    sc_msg(0, "/s_new", ["reverb", 1001, 1, 0, "outbus", 0, "room", 0.7, "mix", 0.1, "damp", 0.9]);
else: pass

In [7]:
if (soundmodel == "vocal") :
    # use test tone to check if the reverb is one
    sc_msg(0, "/s_new", ["default", 1002, 1, 1]); now = time.time(); 
    sc_msg(now+0.5, "/n_free", [1002])
else: pass

OSCClientError: while sending: [Errno 61] Connection refused

## 4. Synth Parameter Definition (for mapping)

In [7]:
# TH: for vocal synth
parspec_vocal = array([ # name, min, max, scaling (lin/exp), default
#("evrate", 0.2, 4, "exp", 0.5, "Hz"),
#("irregularity", 0, 1, "lin", 0, "%"),
("dur", 0.005, 1.5, "exp", 0.4, "secs"), 
("att", 0.001, 0.5, "exp", 0.001, "secs"),
("decslope", -50, 10, "lin", -12, "dB/rm time"),
("amint",  0, 1, "lin", 0, "intensity"),
("amfreq", 1, 50, "exp", 1, "Hz"),
("pitch", 20, 85, "lin", 50, "midinote"),
("chirp", -36, 36, "lin", 0, "semitones/dur"),
("lfnfrq", 5, 50, "exp", 5, "Hz"),
("lfnint", 0, 0.5, "lin", 0, "rel. pitch"),
("vowel", 0, 4, "lin", 2.5, "uoaei"),
("voweldiff", -2.5, 2.5, "lin", 0, "delta"),
("bright", 0.2, 1, "lin", 0.5, "arb.u.")], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

In [8]:
# JJ: New parspec
parspec_abstract = array([ # name, min, max, scaling (lin/exp), default
#("evrate", 0.2, 4, "exp", 0.5, "Hz"),
#("irregularity", 0, 1, "lin", 0, "%"),
("dur", 0., 1., "lin", 0.5, "secs"),
("att", 0., 1., "lin", 0.3, "%"),
("desvol", 0., 1., "lin", 0.5, "dB/dur"),
("pitch", 0, 1., "lin", 0.5, "Hz"),
("chirp", 0., 1., "lin", 0.5, "semitones/dur"),
("lfndepth", 0., 1., "lin", 0., "rate"),
("lfnfreq", 0, 1., "lin", 0., "Hz"),
("amdepth", 0., 1., "lin", 0., "rate"),
("amfreq", 0., 1., "lin", 0., "Hz"),
("richness", 0., 1., "lin", 0.5, "%"),
("lpfreq", 0., 1., "lin", 0.5, "Hz")  # I wonder if this is important.       
    ], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

In [9]:
# JJ: for physio synth
# For the max model, the control range is always between 0 ~ 1. as the scaling is done
# within Max, this is because of the node visulisation in Max which only takes 0 ~ 1.
parspec_physiological = array([ # name, min, max, scaling, default, unit
("bblend", 0., 1., "lin", 0.5, "%"),
("bin", 0., 1., "lin", 0.5, "secs"),
("bgap", 0., 1., "lin", 0.5, "secs"),
("bout", 0, 1., "lin", 0.5, "rate"),
("hbrate", 0., 1., "lin", 0.551, "BPM"),
("hbroughness", 0., 1., "lin", 0.12, "rate"),
("mtvolpit", 0, 1., "lin", 0.73, "dB&Hz"),
("mtfm", 0., 1., "lin", 0., "rate"),
("mtharmonic", 0., 1., "lin", 0., "%"),
("mtroughness", 0., 1., "lin", 0., "rate")      
    ], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

## 5. Mapping Functions and Service Functions for parameter evolution

In [10]:
def parmap(par=("pitch", 20, 85, "lin", 50, "midinote"), val=0.5):
    mi, ma = par[1], par[2]
    if(par[3]=="lin"): return mi+(ma-mi)*val
    if(par[3]=="exp"): return mi*exp(log(ma/mi)*val)

def parunmap(par=("pitch", 20, 85, "lin", 50, "midinote"), val=40):
    mi, ma = par[1], par[2]
    if(par[3]=="lin"): return (val-mi)/(ma-mi)
    if(par[3]=="exp"): return log(val/mi)/log(ma/mi)
    
def parvecmap(parspec, vec):
    return array([parmap(parspec[k], v) for k,v in enumerate(vec)])

def parvecunmap(parspec, vec):
    return array([parunmap(parspec[k], v) for k,v in enumerate(vec)])    

# test code:
# print parvecunmap(parspec_vocal, parspec['default']) # get default parameters

def playevent(soundmodel, v):
    # v is unmapped vector, i.e. vector elements in [0,1]
    if(soundmodel=='vocal'):
        ps = parspec_vocal
        vec = parvecmap(ps, v);
        sc_msg(0, "/s_new", ["jj1", 1002+random.randint(900), 1,1] + 
           [x for pair in zip(ps['name'].tolist(), vec) for x in pair] );
    if(soundmodel=='abstract'):
        ps = parspec_abstract
        vec = parvecmap(ps, v);
        max_msg(0, "/s_new" , [x for pair in zip(ps['name'].tolist(), vec) for x in pair] )
        
    if(soundmodel=='physiological'):
        ps = parspec_physiological
        vec = parvecmap(ps, v);
        max_msg(0, "/s_new" , [x for pair in zip(ps['name'].tolist(), vec) for x in pair] )
        
        

def mutate(parent, sigma=0.1):
    d=size(parent)
    child = clip((parent + sigma*random.randn(d)), 0, 1)
    return child

def create_next_generation(parentvec, sigma, nr_of_children=4):
    global generation_counter, v
    generation_counter += 1
    return [parentvec] + [mutate(parentvec, sigma) for k in range(nr_of_children)]

def append_data(dataset, time, target, generation_counter, logsigma, parvec, submit=0, userid=-1, sound="vocal", run=-1):
    dataset.append([userid, sound, run, time, target, generation_counter, logsigma, parvec, submit])    

def savedata(dataset, prefix="user-soundmodel-run-"):
    df = pandas.DataFrame(data, columns=['uid', 'snd', 'run', 'time','target','generation','logsigma','parvec','submit'])
    df.to_csv(prefix + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)

## 6. Evolutionary Optimization User Interface

In [11]:
username = "fake"
userid = 1010
# soundmodel = "abstract"
run = 1
log_sigma_initval = -0.8
log_sigma_step = -0.2

data = []  # time, target state, generation_counter, sigma, parvec
generation_counter = 0
v = []
target_set_time = time.time()

wlogsigma = widgets.FloatSlider(value=-0.5, min=-5, max=0, step=0.01, description='log(sigma):')

def reset_settings():
    global wtarget, generation_counter, v, wlogsigma, soundmodel
    generation_counter = 0
    target_set_time = time.time()
    if(soundmodel=='vocal'):    ps = parspec_vocal
    if(soundmodel=='abstract'): ps = parspec_abstract
    if (soundmodel == 'physiological'): ps = parspec_physiological
    v = create_next_generation(parvecunmap(ps, ps['default']), 0.25)
    wlogsigma.value = log_sigma_initval
    
reset_settings()

wtarget = widgets.ToggleButtons(
    description='select target',
    options=['happy', 'surprised', 'angry', 'disgusted', 'sad', 'calm'])

def target_on_value_change(change):
    global generation_counter, target_set_time
    generation_counter = 0
    target_set_time = time.time()

wtarget.observe(target_on_value_change, names='value')  

w = widgets.ToggleButtons(
    description='Choose best variation:',
    options=['0', '1', '2', '3', '4'])

def nextgenclick(arg):
    global v, w, wlogsigma
    parent = int(w.value)
    append_data(data, time.time()-target_set_time, wtarget.value, generation_counter, 
                wlogsigma.value, v[parent].tolist(), userid=userid, sound=soundmodel, run=run)
    v = create_next_generation(v[parent], exp(wlogsigma.value))
    wlogsigma.value += log_sigma_step
    w.value = '0'
    
wbutnext = widgets.Button(description='proceed')
wbutnext.on_click(nextgenclick)

wbutsubmit = widgets.Button(description='accept')

def submit_choice(arg):
    global w, v, wlogsigma
    choice = int(w.value)
    print "submit", wtarget.value, choice
    append_data(data, time.time() - target_set_time, wtarget.value, generation_counter, 
                wlogsigma.value, v[choice].tolist(), submit=1, userid=userid, sound=soundmodel, run=run)
    reset_settings()
    w.value = '0'
    targetidx = wtarget.options.index(wtarget.value)
    print targetidx, len(wtarget.options)
    if(targetidx < len(wtarget.options)-1):
        wtarget.value = wtarget.options[targetidx+1]
    else:
        print "completed. thanks."
        fname = "../data/%s-%s-run%d" % (username, soundmodel, run)
        savedata(data, fname)
        print "data saved to"+fname 
    
wbutsubmit.on_click(submit_choice)

display(wtarget, w, wbutnext, wlogsigma, wbutsubmit)

def on_value_change(change):
    global v, wlogsigma
    id = int(change['new'])
    print "play "
    playevent(soundmodel, v[id])
    
w.observe(on_value_change, names='value')  

## 8. Load all data for evaluation

In [ ]:
import glob

def parselist(s):
    return array([ float(el) for el in s.translate(None, "[]").split(",") ], dtype='float64')

converterdict = {'parvec': parselist};

# load all data files of pattern into dataframe
for i, fname in enumerate(glob.glob("../data/*.csv")):
    df = pandas.read_csv(fname, converters=converterdict)
    if(i==0): 
        da = df
    else: 
        da = da.append(df)

In [ ]:
da

## 9. extract all optimization endpoints and play them in a series
(can now be done easier since I added logging for the submit button... to be updated)

In [ ]:
# play all sounds for a given emtion
# soundmodel = 'vocal'
dsel = da[(da['submit']==1) & (da['target']=='disgusted') & (da['snd']==soundmodel)]

for el in dsel.iterrows():
    parvec = el[1]['parvec']
    print el[1]['uid']
    playevent(soundmodel, parvec)
    time.sleep(1.0)    

In [ ]:
# play center of mass of all emotion-assigned samples
targets = ['happy', 'surprised', 'angry', 'disgusted', 'sad', 'calm']
soundmodel = "abstract"
for t in targets:
    print t
    dsel = da[(da['submit']==1) & (da['target']==t) & (da['snd']==soundmodel)]
    vt = mean(dsel['parvec']) 
    playevent(soundmodel, vt)
    time.sleep(1.5)    

## 10. interpolation experiment: play series of sounds between prototypes

In [ ]:
# we need a function to convert a array of arrays into a regular 2D array
def aoa_2d_array(pv):
    r = zeros((size(pv), size(pv[0])))
    for i, el in enumerate(pv): r[i] = el
    return r 
# test code
#soundmodel = 'abstract'
#aoa_2d_array(da[(da['submit']==1) & (da['snd']==soundmodel)].parvec.values)

In [ ]:
# array of all submitted parameter vectors
# soundmodel='vocal'
dtmp = da[(da['submit']==1) & (da['snd']==soundmodel) & 
          (da['uid']==1001) & (da['run']==0)].parvec.values
pvarr = aoa_2d_array(dtmp)
playevent(soundmodel, mean(pvarr, 0)) # mean vector of all submitted prototypes

In [ ]:
dtmp = da[(da['submit']==1) & (da['snd']==soundmodel) & (da['uid']==1001) & (da['run']==0) ]
# print dtmp
par = aoa_2d_array(dtmp['parvec'].values)
# interpolate between emotional prototypes
for l in arange(0, 1, 0.1):
    playevent(soundmodel, l*par[2]+(1.0-l)*par[3])
    time.sleep(1)

## Kernel regression-mapping for navigating between Emotionals Prototypes

In [ ]:
# assemble 6 prototype vectors
dsel = da[(da['submit']==1) & (da['uid']==1001) & (da['snd']=='abstract') & (da['run']==1)]

In [ ]:
targets = ['happy', 'surprised', 'angry', 'disgusted', 'sad', 'calm']
pvec = dsel[:1].parvec.values[0]
dim = len(pvec)
Nrows = len(targets)
pvecs = zeros((Nrows, dim))
for i, t in enumerate(targets):
    print i, t
    pvecs[i]= dsel[dsel['target']==t].parvec.values[0]
    #print t; playevent("jj1", parspec, pvecs[i]); time.sleep(1.5)

# kernel regression: input positions
xvecs = zeros((Nrows, 2))
for i in range(len(targets)):
    xvecs[i] = [cos(2*pi*i/Nrows+0.1), sin(2*pi*i/Nrows+0.1)]

def kernel(x, y, sigma=1):
    return exp(-0.5*sum((x-y)**2)/sigma**2)

def krm(xvecs, pvecs, xvec, sigma=1):
    n=shape(xvecs)[0]
    nom = zeros(dim)
    den = 0
    for i in range(n):
        temp = kernel(xvecs[i], xvec, sigma)
        print temp
        nom += temp*pvecs[i]
        den += temp
    return nom/den
# print xvecs[0], krm(xvecs, pvecs, xvecs[0], sigma=0.1)    
# playevent(soundmodel, krm(xvecs, pvecs, array([-1,1]), sigma=0.9))

In [ ]:
xvecs

In [ ]:
fig = figure()
ax = fig.add_subplot(111)
ax.plot(xvecs[:,0], xvecs[:,1], "o", markersize=15)
ax.set_aspect('equal', 'datalim')

wkrmsigma = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='krm-sigma:')

def onclick(event, verbose=False):
    x = event.xdata; y = event.ydata;
    if(event.button==1): #left mouse click
        if(verbose): print('(%f, %f)' % (x, y))
        playevent(soundmodel, krm(xvecs, pvecs, array([x,y]), sigma=wkrmsigma.value))
cid = fig.canvas.mpl_connect('button_press_event', onclick)
display(wkrmsigma)
get_current_fig_manager().window.setGeometry(1200,10,400,400)

In [ ]:
soundmodel

In [ ]:
# Set up OSC receiver for the tablet interaction. 
soundmodel = 'vocal'
receive_address = '129.70.148.97', 50010 # The receive address needs to be controlled by the tablet
receiveSever = OSC.OSCServer(receive_address) # create a serve to receive OSC from the tablet
receiveSever.addDefaultHandlers()
x = 0.0
y = 0.0
wkrmsigma = 0.8 # later it can be control by tablet. 

# Need to change it to a class. 

def update_OSCAddress(addr,tags,stuff, source):
    global receive_address
    receive_address = stuff[0], int(stuff[1])
    print "New address:" + stuff[0]
    print "New port: "+ stuff[1]
    


def printing_handler(addr, tags, stuff, source):
    global x, y, soundmodel, xvecs, pvecs, wkrmsigma
    x = float(stuff[0])
    y = float(stuff[1])
    print x, y
    playevent(soundmodel, krm(xvecs, pvecs, array([x,y]), sigma=wkrmsigma))

receiveSever.addMsgHandler("/a", printing_handler) # adding our function
receiveSever.addMsgHandler("/addrInfo", update_OSCAddress)
# just checking which handlers we have added
print "Registered Callback-functions are :"
for addr in receiveSever.getOSCAddressSpace():
    print addr
    
# Start OSCServer
print "\nStarting OSCServer."
st_oscSever = threading.Thread( target = receiveSever.serve_forever )
st_oscSever.start()

In [ ]:
# Close the OSC server
print "\nClosing OSCServer."
receiveSever.close()
print "Waiting for Server-thread to finish"
st_oscSever.join() ##!!!
print "Done"